In [57]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [58]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("TEAM_NAME_base", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [59]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/updates/2024_04_03.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)

In [60]:
df['date_next'] = add_col(df, 'Date')
df['date_next'].fillna('2024-04-03', inplace=True)
dates = df['date_next'].unique()

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/2200988528.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("TEAM_NAME_base", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/1205160199.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace

In [61]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [62]:
# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 10:00 AM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

In [63]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
# adjust number of dates 
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings", ""]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

In [64]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

In [65]:
df

,Timestamp,Id,Sportsbook,Team,Odds
0,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,DraftKings,Los Angeles Lakers,1.12
1,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,DraftKings,Toronto Raptors,6.75
2,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,FanDuel,Los Angeles Lakers,1.11
3,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,FanDuel,Toronto Raptors,7.10
4,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,BetMGM,Los Angeles Lakers,1.11
...,...,...,...,...,...
103,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,DraftKings,Phoenix Suns,1.44
104,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,FanDuel,Cleveland Cavaliers,2.94
105,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,FanDuel,Phoenix Suns,1.43
106,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,BetMGM,Cleveland Cavaliers,2.80


In [66]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


[]

In [67]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [68]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/1165239269.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concat = pd.concat([df, missing_df], ignore_index=True)


In [69]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,DraftKings,Los Angeles Lakers,1.12,Toronto Raptors,6.75
1,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,DraftKings,Toronto Raptors,6.75,Los Angeles Lakers,1.12
2,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,FanDuel,Los Angeles Lakers,1.11,Toronto Raptors,7.10
3,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,FanDuel,Toronto Raptors,7.10,Los Angeles Lakers,1.11
4,2024-04-02T13:55:39Z,0d31988221feb992519b909a81ac259b,BetMGM,Los Angeles Lakers,1.11,Toronto Raptors,6.75
...,...,...,...,...,...,...,...
103,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,DraftKings,Phoenix Suns,1.44,Cleveland Cavaliers,2.85
104,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,FanDuel,Cleveland Cavaliers,2.94,Phoenix Suns,1.43
105,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,FanDuel,Phoenix Suns,1.43,Cleveland Cavaliers,2.94
106,2024-04-03T13:55:38Z,e142f1d935ecab70b3e1676a73f130ca,BetMGM,Cleveland Cavaliers,2.80,Phoenix Suns,1.44


In [70]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [71]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [72]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-02,0d31988221feb992519b909a81ac259b,DraftKings,LAL,1.12,TOR,6.75
1,2024-04-02,0d31988221feb992519b909a81ac259b,DraftKings,TOR,6.75,LAL,1.12
2,2024-04-02,0d31988221feb992519b909a81ac259b,FanDuel,LAL,1.11,TOR,7.10
3,2024-04-02,0d31988221feb992519b909a81ac259b,FanDuel,TOR,7.10,LAL,1.11
4,2024-04-02,0d31988221feb992519b909a81ac259b,BetMGM,LAL,1.11,TOR,6.75
...,...,...,...,...,...,...,...
103,2024-04-03,e142f1d935ecab70b3e1676a73f130ca,DraftKings,PHO,1.44,CLE,2.85
104,2024-04-03,e142f1d935ecab70b3e1676a73f130ca,FanDuel,CLE,2.94,PHO,1.43
105,2024-04-03,e142f1d935ecab70b3e1676a73f130ca,FanDuel,PHO,1.43,CLE,2.94
106,2024-04-03,e142f1d935ecab70b3e1676a73f130ca,BetMGM,CLE,2.80,PHO,1.44


In [73]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [74]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [75]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [76]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2024-04-02,LAL,1.11,1.12,1.11,TOR,7.10,6.75,6.75
1,2024-04-02,TOR,7.10,6.75,6.75,LAL,1.11,1.12,1.11
2,2024-04-02,CLE,1.12,1.11,1.11,UTA,6.90,7.00,7.00
3,2024-04-02,UTA,6.90,7.00,7.00,CLE,1.12,1.11,1.11
4,2024-04-02,LAC,1.74,1.74,1.71,SAC,2.16,2.14,2.15
5,2024-04-02,SAC,2.16,2.14,2.15,LAC,1.74,1.74,1.71
6,2024-04-02,MIA,1.68,1.68,1.67,NYK,2.24,2.24,2.25
7,2024-04-02,NYK,2.24,2.24,2.25,MIA,1.68,1.68,1.67
8,2024-04-02,DAL,1.96,1.91,1.95,GSW,1.89,1.91,1.87
9,2024-04-02,GSW,1.89,1.91,1.87,DAL,1.96,1.91,1.95


In [77]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
whole = pd.read_csv(file_path, index_col=0)

In [78]:
complete = pd.concat([whole, final], axis=0)

In [79]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
complete.to_csv(file_path)